# Mag 7 Market Capitalization vs. Major Indexes
© 2026 Jim Domeij

Get current market cap (bn USD) for Magnificient 7 companies and other major stock indices. This comparison highlights the dominance of a small number of large firms in global equity markets and the associated concentration risk

In [1]:
import json
from pathlib import Path

import altair as alt

alt.renderers.set_embed_options(actions=False)
chart_file = Path("docs/examples/equity") / "mag7-market-cap-comparison-hero.json"
with open(chart_file) as f:
    spec = json.load(f)
alt.Chart.from_dict(spec)

alt.LayerChart(...)

In [2]:
from polars_bloomberg import BQuery

query = """
    let(
        #name = if(security_typ=='Common Stock', name(), id());
        #mkt_cap = cur_mkt_cap(currency=USD)/1e9;
    )
    get(#name, #mkt_cap)
    for([
        'AAPL US Equity',
        'AMZN US Equity',
        'GOOGL US Equity',
        'META US Equity',
        'MSFT US Equity',
        'NVDA US Equity',
        'TSLA US Equity',
        'SXXP Index',
        'DAX Index',
        'UKX Index',
        'CAC Index',
        'TPX Index',
        'SHCOMP Index'
    ])
    preferences(dropCols=ALL)
"""

with BQuery() as bq:
    df = bq.bql(query).combine()
df.head()

ID,#name,#mkt_cap
str,str,f64
"""AAPL US Equity""","""Apple Inc""",3808.309621
"""AMZN US Equity""","""Amazon.com Inc""",2620.118506
"""GOOGL US Equity""","""Alphabet Inc""",3971.064028
"""META US Equity""","""Meta Platforms Inc""",1636.967446
"""MSFT US Equity""","""Microsoft Corp""",3528.075367


In [3]:
# Chart
import altair as alt
import polars as pl

MAG7_IDS = [
    "AAPL US Equity",
    "MSFT US Equity",
    "AMZN US Equity",
    "GOOGL US Equity",
    "META US Equity",
    "NVDA US Equity",
    "TSLA US Equity",
]

# --- Mag 7 colors
MAG7_COLORS = {
    "TSLA US Equity": "#1D3B5A",  # Navy (base)
    "META US Equity": "#FF5B2F",  # Red
    "AMZN US Equity": "#4885C4",  # Blue
    "MSFT US Equity": "#48B866",  # Green
    "AAPL US Equity": "#FBC111",  # Yellow (center highlight)
    "GOOGL US Equity": "#005D34",  # Dark green
    "NVDA US Equity": "#2AA6A1",  # Teal (top)
}


INDEX_COLOR = "#D3C4A6"  # Beige

# --- Build plotting df in Polars
plot_df = df.with_columns(
    [
        pl.when(pl.col("ID").is_in(MAG7_IDS))
        .then(pl.lit("Magnificent 7"))
        .otherwise(pl.col("#name"))
        .str.replace(" Index", "")
        .alias("basket"),
        pl.when(pl.col("ID").is_in(MAG7_IDS))
        .then(pl.col("#name"))
        .otherwise(pl.lit("Index total"))
        .alias("component"),
        pl.col("#mkt_cap").cast(pl.Float64).alias("mkt_cap"),
    ]
)

# Totals for x-order (Altair 6 layered sorting reliability)
totals = plot_df.group_by("basket").agg(pl.sum("mkt_cap").alias("total_mkt_cap"))
basket_order = (
    totals.sort("total_mkt_cap", descending=True).get_column("basket").to_list()
)

x_sorted = alt.X("basket:N", sort=basket_order, title=None)

# -------------------------
# Bars (ONLY change: colors for Mag7 components, beige for index bars)
# -------------------------
bars = (
    alt.Chart(plot_df)
    .mark_bar()
    .encode(
        x=x_sorted,
        y=alt.Y(
            "sum(mkt_cap):Q", title="Market cap (USD bn)", axis=alt.Axis(format=",.0f")
        ),
        color=alt.condition(
            alt.datum.basket == "Magnificent 7",
            alt.Color(
                "ID:N",
                scale=alt.Scale(
                    domain=list(MAG7_COLORS.keys()),
                    range=list(MAG7_COLORS.values()),
                ),
                legend=None,
            ),
            alt.value(INDEX_COLOR),
        ),
        order=alt.Order("mkt_cap:Q", sort="ascending"),
        tooltip=[
            alt.Tooltip("basket:N", title="Bar"),
            alt.Tooltip("component:N", title="Component"),
            alt.Tooltip("sum(mkt_cap):Q", title="Market cap (USD bn)", format=",.0f"),
        ],
    )
)

# -------------------------
# Labels (Magnificent 7 only) - compute segment midpoints explicitly
# -------------------------
labels = (
    alt.Chart(plot_df)
    .transform_filter(alt.datum.basket == "Magnificent 7")
    .transform_aggregate(seg_cap="sum(mkt_cap)", groupby=["basket", "component"])
    .transform_window(
        cum="sum(seg_cap)",
        groupby=["basket"],
        sort=[{"field": "seg_cap", "order": "ascending"}],
    )
    .transform_calculate(
        y_mid="datum.cum - datum.seg_cap/2", short="split(datum.component, ' ')[0]"
    )
    .mark_text(baseline="middle", fontSize=11, color="black")
    .encode(
        x=x_sorted,
        y=alt.Y("y_mid:Q", title="Market cap (USD bn)"),
        text="short:N",
        tooltip=[
            alt.Tooltip("component:N", title="Company"),
            alt.Tooltip("seg_cap:Q", title="Market cap (USD bn)", format=",.0f"),
        ],
    )
)

chart = (
    alt.layer(bars, labels)
    .resolve_scale(x="shared", y="shared")
    .properties(
        width=600, height=400, title="Mag 7 Market Capitalization vs. Major Indexes"
    )
    .configure_axis(titleFontSize=12, titlePadding=10, labelAngle=0)
)

# Save the chart
chart.save(chart_file)
chart.save(chart_file.with_suffix(".png"), scale_factor=0.75)
